In [1]:
from openmm import app
import openmm as mm
from simtk import unit
from sys import stdout, exit, stderr
from parmed.charmm import CharmmPsfFile, CharmmCrdFile, CharmmParameterSet
from parmed.openmm import StateDataReporter
from parmed import unit as u
import mdtraj as md
#import nglview as ngl

In [2]:
ls

ala_6_drude/          ser_6_drude/       ser_6.pdb*
par_all36m_prot.prm*  ser_6_init.ipynb*  ser_6.psf*


In [4]:
psf = CharmmPsfFile('ser_6.psf')
pdb = app.PDBFile('ser_6.pdb')
params = CharmmParameterSet('par_all36m_prot.prm')
system = psf.createSystem(params, nonbondedMethod=app.NoCutoff,
        nonbondedCutoff=1*u.nanometer, constraints=app.HBonds, implicitSolvent=app.HCT,
        solventDielectric=78.5)
integrator = mm.LangevinIntegrator(298.15*unit.kelvin, 5.0/unit.picoseconds, 2.0*unit.femtoseconds)
integrator.setConstraintTolerance(1e-5)
platform = mm.Platform.getPlatformByName('Reference')
simulation = app.Simulation(pdb.topology, system, integrator, platform)
simulation.context.setPositions(pdb.positions)

In [5]:
print('Minimizing...')

st = simulation.context.getState(getPositions=True,getEnergy=True)
print(F"Potential energy before minimization is {st.getPotentialEnergy()}")

simulation.minimizeEnergy(maxIterations=100)

st = simulation.context.getState(getPositions=True,getEnergy=True)
print(F"Potential energy after minimization is {st.getPotentialEnergy()}")

Minimizing...
Potential energy before minimization is 54.55859822109824 kJ/mol
Potential energy after minimization is -161.2055879023079 kJ/mol


In [6]:
from sys import stdout

print('Equilibrating...')

simulation.reporters.append(app.StateDataReporter(stdout, 250, step=True, 
    potentialEnergy=True, temperature=True, separator=','))
simulation.context.setVelocitiesToTemperature(150.0*unit.kelvin)
simulation.step(5000)

Equilibrating...
#"Step","Potential Energy (kJ/mole)","Temperature (K)"
250,22.5285920848288,210.46155811091995
500,59.76208878519458,304.3494377591646
750,80.03388666996466,329.8817412876106
1000,49.70531369477919,279.1875256192682
1250,36.32295134511253,319.35853872931017
1500,62.336865698255906,267.7189383715732
1750,49.892432265524235,270.9590965188596
2000,19.99088081577247,324.1720903503954
2250,61.617573405768894,308.5999803913977
2500,69.19705874360136,231.01221864527477
2750,28.108653415740832,325.9001662492238
3000,81.78013529353666,295.97764746495085
3250,28.133557803962958,332.71423563602457
3500,38.11260211133299,313.6307978063079
3750,35.40503087291279,304.99002024500487
4000,29.336750725357774,238.84494891129984
4250,21.771094921642657,317.1277158975708
4500,48.71051913894701,363.7690777655111
4750,-4.763204252096216,258.12333319833635
5000,95.45901061602774,316.14655043242317


In [7]:
import time as time

print('Running Production')

# begin timer
tinit=time.time()

# clear simulation reporters
simulation.reporters.clear()

# reinitialize simulation reporters. Done because we want different information printed from the production run
# than the equilibriation run. 
# output basic simulation information below every 50,000 steps - 2 x 50,000 = 100,000fs or 0.1ns
simulation.reporters.append(app.StateDataReporter(stdout, 10000, step=True, time=True, potentialEnergy=True,
                            temperature=True, speed=True, separator=','))

# write traj to a DCD file every 100 steps - 0.2ps
simulation.reporters.append(app.DCDReporter('ser_6_sim.dcd', 100))

# run simulation for 2.5x10^6 steps - 5ns
simulation.step(2500000)

# end timer
tfinal=time.time()
print('Done!')
print('Time required for simulation:', tfinal-tinit, 'seconds')

Running Production
#"Step","Time (ps)","Potential Energy (kJ/mole)","Temperature (K)","Speed (ns/day)"
10000,19.999999999999794,69.3928114282894,316.08423536345936,0
20000,40.00000000000292,51.90963028333067,237.72764633196732,106
30000,60.00000000002736,47.42345578140714,295.46245528290876,106
40000,79.99999999999496,84.46867597090431,284.4796601864262,106
50000,99.99999999994834,43.946589376344605,308.2586849363298,103
60000,119.99999999990173,107.43939518241814,263.75166966644275,102
70000,139.99999999994037,54.02630215288423,289.5450183491702,101
80000,160.00000000003587,42.46970291302273,271.9182067317636,100
90000,180.00000000013137,71.61929138801167,316.106772379967,99.6
100000,200.00000000022686,81.24184645615651,269.69947509210067,99.2
110000,220.00000000032236,53.02440092783445,331.0219584433074,98.8
120000,240.00000000041786,56.83966182095901,326.56482671212933,98.6
130000,260.00000000051335,77.56815088099506,297.1613972251223,98.4
140000,280.00000000060885,59.82461195152393

1220000,2439.9999999561255,85.71075573919791,315.24546753739776,96.2
1230000,2459.9999999556526,-5.4466795710485485,331.05378137422775,96.2
1240000,2479.9999999551796,47.52978609437275,346.9860096935911,96.2
1250000,2499.9999999547067,53.626018717572265,279.5060608800509,96.2
1260000,2519.9999999542338,84.85324489507207,239.98453193598493,96.2
1270000,2539.999999953761,58.86438582996038,314.95636123476055,96.2
1280000,2559.999999953288,53.17467488943703,293.00902378285303,96.2
1290000,2579.999999952815,93.33002750120943,284.1922131642418,96.2
1300000,2599.999999952342,83.82899507468096,235.52685613578168,96.2
1310000,2619.999999951869,89.62853747383429,257.9177764380377,96.2
1320000,2639.999999951396,67.83832470657478,299.91102726708914,96.2
1330000,2659.999999950923,44.81000082602486,329.7817285713962,96.2
1340000,2679.9999999504503,51.590360496805715,337.0113180088475,96.2
1350000,2699.9999999499773,77.08038567781989,269.5308302491998,96.2
1360000,2719.9999999495044,81.09057156720007

2420000,4840.000000068539,27.824956427550774,271.8828430798026,96.4
2430000,4860.000000072613,1.4336655894219348,292.07974538587763,96.4
2440000,4880.000000076688,15.082100738222266,257.8586274568139,96.4
2450000,4900.000000080762,10.542853993016138,346.0734792935952,96.4
2460000,4920.000000084837,-6.280998231105173,291.65427973539755,96.4
2470000,4940.000000088911,15.353888379068962,300.96094866025135,96.4
2480000,4960.000000092986,38.23558076078922,271.70884265398195,96.4
2490000,4980.00000009706,80.7738335872657,288.4813262571946,96.4
2500000,5000.000000101135,64.21254310401355,297.69065353234265,96.4
Done!
Time required for simulation: 4481.755071640015 seconds


In [11]:
#traj = md.load('ala_6_sim.dcd', top='ala_6.pdb')

In [12]:
#visualize = ngl.show_mdtraj(traj)
#visualize

NGLWidget(max_frame=24999)